# Gradient Boosting model

## Import data and set up

In [1]:
# import relevant packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [2]:
# set up paths
data_dir = "../data"

In [3]:
# import data 
X_train = pd.read_csv(os.path.join(data_dir, 'X_train.csv'))
X_test = pd.read_csv(os.path.join(data_dir, 'X_test.csv'))
y_train = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))
y_test = pd.read_csv(os.path.join(data_dir, 'y_test.csv'))

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(397900, 24)
(99476, 24)
(397900, 1)
(99476, 1)


In [4]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [5]:
X_train.columns

Index(['Date', 'Store', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Month', 'Year',
       'Weekday', 'Holiday', 'StoreType_enc', 'Assortment_enc', 'Store_enc'],
      dtype='object')

In [6]:
train_cols = ['Open', 'Promo', 'SchoolHoliday', 'Month', 'Year',
       'Weekday', 'Holiday', 'StoreType_enc', 'Assortment_enc', 'Store_enc']

X_train = X_train[train_cols]
X_test = X_test[train_cols]
print(X_train.columns)
print(X_test.columns)

Index(['Open', 'Promo', 'SchoolHoliday', 'Month', 'Year', 'Weekday', 'Holiday',
       'StoreType_enc', 'Assortment_enc', 'Store_enc'],
      dtype='object')
Index(['Open', 'Promo', 'SchoolHoliday', 'Month', 'Year', 'Weekday', 'Holiday',
       'StoreType_enc', 'Assortment_enc', 'Store_enc'],
      dtype='object')


In [7]:
y_train = y_train.to_numpy().flatten()
y_test = y_test.to_numpy().flatten()

## Gradient Boosting

In [8]:
#xgboost
model = xgb.XGBRegressor(n_jobs=1, random_state=42)

In [17]:
# define parameter space for grid search
parameter_space = {
    'n_estimators': [500],
    'eta': [0.1, 0.05],
    'subsample': [0.7],
    'colsample_bytree': [0.8],
    'max_depth': [5, 7]
}

# n_estimators = number of trees in the foreset
# max_features = max number of features considered for splitting a node
# max_depth = max number of levels in each decision tree
# min_samples_split = min number of data points placed in a node before the node is split
# min_samples_leaf = min number of data points allowed in a leaf node
# bootstrap = method for sampling data points (with or without replacement)

In [18]:
my_scorer = make_scorer(metric, greater_is_better=False)

In [19]:
%%time
regr = GridSearchCV(model, parameter_space, scoring=my_scorer, 
                   n_jobs=None, cv=5, return_train_score = True)
regr.fit(X_train, y_train)

CPU times: user 32min 26s, sys: 2.68 s, total: 32min 29s
Wall time: 1h 46min 47s


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=1,
                                    num_parallel_tree=None, random_state=42,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameter

In [20]:
#Best estimator
print('Best estimator:\n', regr.best_estimator_)

# Best parameter setting
print('Best parameters found:\n', regr.best_params_)

# Scorer used on the held out data to choose best parameters for the model
print('Scorer used on the held out data to choose the best parameters for the model:', regr.scorer_)

# Mean cross-validated score of the best estimator
print('Best mean cross-validated score:', regr.best_score_)

Best estimator:
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)
Best parameters found:
 {'colsample_bytree': 0.8, 'eta': 0.1, 'max_depth': 7, 'n_estimators': 500, 'subsample': 0.7}
Scorer used on the held out data to choose the best parameters for the model: make_scorer(metric, greater_is_better=False)
Best mean cross-validated score: -15.47276144544223


In [ ]:
# All results
cv_results = pd.DataFrame.from_dict(regr.cv_results_)
cv_results

In [21]:
y_pred = regr.predict(X_test)
value = metric(y_pred, y_test)
print(value)

18.862475261226205
